# EDA das ofertas relâmpagos

## Objetivo

* Como entender como as variáveis (*features*) se relacionam com as *ofertas relâmpagos*.
  * **Estratégia**: Análise univariada e bivariada (principal) - distribuição conjunta de duas variáveis.

In [32]:
#Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt   
##Eliminar os warnings
import warnings
warnings.filterwarnings("ignore")
##Ver todas as colunas do data frame
pd.set_option('display.max_columns', None)
##Ver todas as linhas do data frame
pd.set_option('display.max_rows', None)
##Dados sem notação científica
pd.set_option('display.float_format', '{:,.2f}'.format)

# 1- Visão preliminar dos dados

In [2]:
#Importar a base de dados
df =  pd.read_csv('ofertas_relampago.csv', sep = ',')
#Visualizar
df.head(3) #Visualizar cada linha como uma promoção

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS


In [3]:
#Detalhamento das variáveis
df.info()   #Volumetria: 48746 observações
            #Colunas (variáveis): 12

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48746 entries, 0 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OFFER_START_DATE           48746 non-null  object 
 1   OFFER_START_DTTM           48746 non-null  object 
 2   OFFER_FINISH_DTTM          48746 non-null  object 
 3   OFFER_TYPE                 48746 non-null  object 
 4   INVOLVED_STOCK             48746 non-null  int64  
 5   REMAINING_STOCK_AFTER_END  48746 non-null  int64  
 6   SOLD_AMOUNT                24579 non-null  float64
 7   SOLD_QUANTITY              24579 non-null  float64
 8   ORIGIN                     11316 non-null  object 
 9   SHIPPING_PAYMENT_TYPE      48746 non-null  object 
 10  DOM_DOMAIN_AGG1            48746 non-null  object 
 11  VERTICAL                   48746 non-null  object 
 12  DOMAIN_ID                  48746 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usag

## Entendendo as variáveis

In [4]:
#Variável - OFFER_START_DATE	
df['OFFER_START_DATE'].drop_duplicates().shape[0] #61 dias de oferta relâmpago

61

### Variáveis categóricas

In [5]:
#Variável - OFFER_TYPE 
df['OFFER_TYPE'].value_counts() #Todas foram lightning_deal (oferta relâmpago) - esta variável pode ser descartada no estudo descritivo

OFFER_TYPE
lightning_deal    48746
Name: count, dtype: int64

In [6]:
#Variável - ORIGIN
df['ORIGIN'].value_counts() #Classe A foi a mais frequente (Variável com alta incidência de missing values)

ORIGIN
A    11316
Name: count, dtype: int64

In [7]:
#Variável - SHIPPING_PAYMENT_TYPE
df['SHIPPING_PAYMENT_TYPE'].value_counts() #free_shipping (frete grátis) vs none (frete pago)

SHIPPING_PAYMENT_TYPE
free_shipping    26658
none             22088
Name: count, dtype: int64

In [8]:
#Variável - DOM_DOMAIN_AGG1 --Tipo de produto comercializado (várias classes)
round(100 * df['DOM_DOMAIN_AGG1'].value_counts(True),1).head() 

DOM_DOMAIN_AGG1
HOME&DECOR            13.8
APPAREL ACCESORIES     9.0
APPAREL                7.9
COMPUTERS              7.7
SPORTS                 6.7
Name: proportion, dtype: float64

* **Classes com mais representatividade (%)**
1. HOME&DECOR                      13.8
2. APPAREL ACCESORIES               9.0 (ACESSÓRIOS DE VESTUÁRIO)
3. APPAREL                          7.9
4. COMPUTERS                        7.7
5. SPORTS                           6.7
6. PHARMACEUTICS                    6.2
7. ELECTRONICS                      6.2
8. PERSONAL CARE                    6.2

In [10]:
#Estas 8 classes tem aproximadamente 64% do total de casos
round(13.8 + 9.0 + 7.9 + 7.7 + 6.7 + 6.2 + 6.2 + 6.2,0) 

64.0

In [14]:
#Variável - VERTICAL
round(100 * df['VERTICAL'].value_counts(True),1) #Pode ser visto como uma agregação da variável DOM_DOMAIN_AGG1

VERTICAL
APP & SPORTS       27.2
HOME & INDUSTRY    22.6
CE                 17.7
BEAUTY & HEALTH    14.7
CPG                 5.6
T & B               5.4
ACC                 5.3
ENTERTAINMENT       1.1
OTHERS              0.6
Name: proportion, dtype: float64

In [15]:
#As classes APP & SPORTS, HOME & INDUSTRY, CE e BEAUTY & HEALTH 
27.2 + 22.6 + 17.7 + 14.7 #82.2% dos dados

82.2

In [16]:
#Variável - DOMAIN_ID
df['DOMAIN_ID'].value_counts().shape[0] #1266 (muitas classes)

1266

In [17]:
#Variável - DOMAIN_ID
df['DOMAIN_ID'].value_counts().head() #Principal: MLM-SURGICAL_AND_INDUSTRIAL_MASKS

DOMAIN_ID
MLM-SURGICAL_AND_INDUSTRIAL_MASKS    1759
MLM-HEADPHONES                        761
MLM-LED_STRIPS                        518
MLM-SUNGLASSES                        472
MLM-VEHICLE_LED_BULBS                 454
Name: count, dtype: int64

# 2 - Data wrangling & Feature Engineering

In [18]:
#OFFER_START_DATE no formato de data
df['OFFER_START_DATE'] = pd.to_datetime(df.OFFER_START_DATE, format = '%Y-%m-%d')

In [19]:
#Criar a variável SAFRA: Mês da promoção
df['SAFRA'] = df['OFFER_START_DATE'].dt.to_period('M')
#Casting para string
df = df.astype({'SAFRA': 'string'})

In [20]:
#Converte as colunas para datetime
df['OFFER_START_DTTM'] = pd.to_datetime(df['OFFER_START_DTTM'])
df['OFFER_FINISH_DTTM'] = pd.to_datetime(df['OFFER_FINISH_DTTM'])

In [21]:
#Criar a variável DURACAO, duração da promoção (medido em horas)
df['DURACAO'] = ((df['OFFER_FINISH_DTTM'] - df['OFFER_START_DTTM']).dt.total_seconds() / 3600).round(2)

In [22]:
#Criar a variável DIA_SEMANA
dias_semana_traducao = {
    'Monday': 'segunda',
    'Tuesday': 'terca',
    'Wednesday': 'quarta',
    'Thursday': 'quinta',
    'Friday': 'sexta',
    'Saturday': 'sabado',
    'Sunday': 'domingo'
}

df['DIA_SEMANA'] = df['OFFER_START_DTTM'].dt.day_name().map(dias_semana_traducao)

In [23]:
#FLAG_N_VENDEU: 1 se SOLD_AMOUNT é nulo, 0 caso contrário
df['FLAG_N_VENDEU'] = np.where(df['SOLD_AMOUNT'].isna(), 1, 0).astype(int)

In [24]:
#Criar a variável VENDA_POR_HORA com duas casas decimais - quanto maior o valor da variável, maior a saída do produto
df['VENDA_POR_HORA'] = (df['SOLD_AMOUNT'] / df['DURACAO']).round(2)

In [26]:
#FLAG_ESTOQUE_NEGATIVO: 1 se o estoque final for negativo, 0 caso contrário
df['FLAG_ESTOQUE_NEGATIVO'] = (df['REMAINING_STOCK_AFTER_END'] < 0).astype(int)

In [27]:
#Criar a variável RECEITA_PROMOCAO
df['RECEITA_PROMOCAO'] = (df['SOLD_AMOUNT'] * df['SOLD_QUANTITY']).fillna(0)

In [28]:
#Visualização
df.head(3) #7 variáveis foram criadas (Gera um CSV com as variáveis criadas)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,SAFRA,DURACAO,DIA_SEMANA,FLAG_N_VENDEU,VENDA_POR_HORA,FLAG_ESTOQUE_NEGATIVO,RECEITA_PROMOCAO
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS,2021-06,7.05,terca,0,0.67,1,28.32
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS,2021-06,6.00,terca,1,NaN,0,0.00
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS,2021-06,6.00,terca,0,1.79,0,32.19


# 3 - EDA

In [41]:
#Volume de promoções por dia da semana
round(100 * df['DIA_SEMANA'].value_counts(True),1) 

DIA_SEMANA
sexta     15.90
sabado    15.40
terca     15.00
quinta    14.90
quarta    13.80
segunda   12.90
domingo   12.00
Name: proportion, dtype: float64

In [43]:
vendas_por_dia = df.groupby('DIA_SEMANA', as_index=False).agg({
    'SOLD_QUANTITY': 'mean',
    'RECEITA_PROMOCAO': 'mean'
}).round(2)

# Renomeando as colunas
vendas_por_dia.rename(columns={
    'SOLD_QUANTITY': 'QUANTIDADE_MEDIA_VENDIDA_DIA',
    'RECEITA_PROMOCAO': 'RECEITA_MEDIA_VENDIDA_DIA'
}, inplace=True)

vendas_por_dia

,DIA_SEMANA,QUANTIDADE_MEDIA_VENDIDA_DIA,RECEITA_MEDIA_VENDIDA_DIA
0,domingo,7.09,831.64
1,quarta,12.23,"4,227.72"
2,quinta,11.15,"2,739.53"
3,sabado,8.33,"1,437.12"
4,segunda,13.54,"5,754.72"
5,sexta,9.95,"2,275.72"
6,terca,12.99,"4,715.82"


In [29]:
#Meses disponíveis
df['SAFRA'].drop_duplicates() #Meses de junho e julho de 2021

0      2021-06
847    2021-07
Name: SAFRA, dtype: string

In [35]:
#Volumetria de promoções por mês 
df['SAFRA'].value_counts().sort_index()

SAFRA
2021-06    18336
2021-07    30410
Name: count, dtype: Int64

In [33]:
#Quantidade vendida e receita nos meses (absoluto)
vendas_por_mes = df.groupby('SAFRA', as_index=False).agg({
    'SOLD_QUANTITY': 'sum',
    'RECEITA_PROMOCAO': 'sum'
})

vendas_por_mes

,SAFRA,SOLD_QUANTITY,RECEITA_PROMOCAO
0,2021-06,"87,726.00","31,212,125.51"
1,2021-07,"178,982.00","121,302,214.36"


**Produtos que não venderam na promoção**

In [36]:
#Não vendeu por safra
nao_vendeu_por_safra = df.groupby('SAFRA', as_index=False)['FLAG_N_VENDEU'].sum()
nao_vendeu_por_safra

,SAFRA,FLAG_N_VENDEU
0,2021-06,9406
1,2021-07,14761


In [37]:
n_vendeu = df.loc[df.FLAG_N_VENDEU ==1]
#Visualização
n_vendeu.head(2)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,SAFRA,DURACAO,DIA_SEMANA,FLAG_N_VENDEU,VENDA_POR_HORA,FLAG_ESTOQUE_NEGATIVO,RECEITA_PROMOCAO
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS,2021-06,6.00,terca,1,NaN,0,0.00
5,2021-06-22,2021-06-22 18:00:00+00:00,2021-06-23 01:00:12+00:00,lightning_deal,23,23,NaN,NaN,A,free_shipping,COMPUTERS,CE,MLM-HEADPHONES,2021-06,7.00,terca,1,NaN,0,0.00


In [38]:
n_vendeu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24167 entries, 1 to 48745
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   OFFER_START_DATE           24167 non-null  datetime64[ns]     
 1   OFFER_START_DTTM           24167 non-null  datetime64[ns, UTC]
 2   OFFER_FINISH_DTTM          24167 non-null  datetime64[ns, UTC]
 3   OFFER_TYPE                 24167 non-null  object             
 4   INVOLVED_STOCK             24167 non-null  int64              
 5   REMAINING_STOCK_AFTER_END  24167 non-null  int64              
 6   SOLD_AMOUNT                0 non-null      float64            
 7   SOLD_QUANTITY              0 non-null      float64            
 8   ORIGIN                     6108 non-null   object             
 9   SHIPPING_PAYMENT_TYPE      24167 non-null  object             
 10  DOM_DOMAIN_AGG1            24167 non-null  object             
 11  VERTICA

In [39]:
#Variável - VERTICAL
round(100 * n_vendeu['VERTICAL'].value_counts(True),1) 

VERTICAL
APP & SPORTS      27.30
HOME & INDUSTRY   23.30
CE                17.00
BEAUTY & HEALTH   11.30
T & B              7.60
CPG                6.20
ACC                5.20
ENTERTAINMENT      1.40
OTHERS             0.70
Name: proportion, dtype: float64

In [40]:
#Variável - VERTICAL
round(100 * n_vendeu['DIA_SEMANA'].value_counts(True),1) 

DIA_SEMANA
sabado    16.60
sexta     16.30
terca     14.90
quinta    14.50
quarta    12.90
segunda   12.40
domingo   12.30
Name: proportion, dtype: float64